In [1]:
!pip install -q gradio pandas scikit-learn


Simulate Logs (JSON format)

In [2]:
import pandas as pd
import random
import json

# Simulate 100 system logs
levels = ["INFO", "WARNING", "ERROR", "CRITICAL"]
services = ["auth", "network", "db", "app"]

logs = []
for _ in range(100):
    logs.append({
        "timestamp": pd.Timestamp.now(),
        "level": random.choices(levels, weights=[60, 20, 15, 5])[0],
        "service": random.choice(services),
        "user_id": random.randint(1, 50),
        "latency": round(random.gauss(100, 20), 2),
        "bytes": random.randint(100, 5000)
    })

df_logs = pd.DataFrame(logs)
df_logs.head()


,timestamp,level,service,user_id,latency,bytes
0,2025-07-14 21:37:18.412957,INFO,auth,13,94.47,2123
1,2025-07-14 21:37:18.413218,WARNING,db,17,101.14,1794
2,2025-07-14 21:37:18.413244,WARNING,db,1,89.54,4364
3,2025-07-14 21:37:18.413267,WARNING,app,10,86.18,797
4,2025-07-14 21:37:18.413299,INFO,app,46,94.45,1511


Preprocess and Engineer Features

In [3]:
# Feature Engineering
df_logs["level_num"] = df_logs["level"].map({
    "INFO": 0, "WARNING": 1, "ERROR": 2, "CRITICAL": 3
})

df_logs["service_code"] = df_logs["service"].astype("category").cat.codes

# Features
X = df_logs[["latency", "bytes", "level_num", "service_code"]]


Train Anomaly Detection AI

In [4]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(contamination=0.1, random_state=42)
df_logs["anomaly"] = model.fit_predict(X)

# Map output to labels
df_logs["anomaly_label"] = df_logs["anomaly"].map({1: "Normal", -1: "Suspicious"})

df_logs[["timestamp", "service", "level", "anomaly_label"]].tail(10)


,timestamp,service,level,anomaly_label
90,2025-07-14 21:37:18.414449,auth,ERROR,Normal
91,2025-07-14 21:37:18.414462,auth,INFO,Normal
92,2025-07-14 21:37:18.414473,auth,INFO,Normal
93,2025-07-14 21:37:18.414487,network,CRITICAL,Suspicious
94,2025-07-14 21:37:18.414499,network,INFO,Suspicious
95,2025-07-14 21:37:18.414511,network,INFO,Normal
96,2025-07-14 21:37:18.414524,auth,INFO,Normal
97,2025-07-14 21:37:18.414536,app,INFO,Normal
98,2025-07-14 21:37:18.414547,auth,INFO,Normal
99,2025-07-14 21:37:18.414559,app,ERROR,Normal


radio Live Dashboard

In [5]:
import gradio as gr

def analyze_log(latency, bytes_sent, level, service):
    # Convert to features
    level_map = {"INFO": 0, "WARNING": 1, "ERROR": 2, "CRITICAL": 3}
    service_map = {name: i for i, name in enumerate(df_logs["service"].unique())}

    features = [[latency, bytes_sent, level_map[level], service_map[service]]]
    pred = model.predict(features)[0]
    return "🔒 Normal" if pred == 1 else "🚨 Suspicious Activity Detected!"

iface = gr.Interface(
    fn=analyze_log,
    inputs=[
        gr.Slider(0, 300, label="Latency (ms)"),
        gr.Slider(100, 5000, label="Bytes Sent"),
        gr.Radio(["INFO", "WARNING", "ERROR", "CRITICAL"], label="Log Level"),
        gr.Dropdown(df_logs["service"].unique().tolist(), label="Service")
    ],
    outputs=gr.Textbox(),
    title="🛡️ AI Log Analyzer",
    description="Enter system log values to detect suspicious activity using anomaly detection."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ecba9d220a47be0d01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
